In [50]:
# Cell 1: Imports and functions
import pandas as pd
import pyodbc
import re
import sys
from decimal import Decimal

def normalize_name(s: str) -> str:
    return re.sub(r'[^0-9a-z]', '', s.lower())

# SQL type groups
_INT_TYPES   = {'tinyint', 'smallint', 'int', 'bigint', 'bit'}
_FLOAT_TYPES = {'float', 'real'}
_DEC_TYPES   = {'decimal', 'numeric'}
_DATE_TYPES  = {'date', 'datetime', 'datetime2', 'smalldatetime', 'time'}

def append_csv_to_sqlserver(
    csv_path: str,
    table_name: str,
    server: str,
    database: str,
    username: str,
    password: str,
    dayfirst: bool = True
):
    # 1) Read all as strings
    df = pd.read_csv(csv_path, dtype=str).fillna('')

    # 2) Get schema info
    conn_str = (
        "DRIVER={ODBC Driver 17 for SQL Server};"
        f"SERVER={server};DATABASE={database};"
        f"UID={username};PWD={password}"
    )
    conn = pyodbc.connect(conn_str, timeout=5)
    cursor = conn.cursor()

    if '.' in table_name:
        schema, tbl = table_name.split('.', 1)
    else:
        schema, tbl = 'dbo', table_name

    cursor.execute("""
      SELECT 
        COLUMN_NAME, DATA_TYPE, 
        NUMERIC_PRECISION, NUMERIC_SCALE
      FROM INFORMATION_SCHEMA.COLUMNS
      WHERE TABLE_SCHEMA = ? AND TABLE_NAME = ?
    """, schema, tbl)
    cols_info = [
        {
            'name': row.COLUMN_NAME,
            'dtype': row.DATA_TYPE.lower(),
            'prec': row.NUMERIC_PRECISION,
            'scale': row.NUMERIC_SCALE
        }
        for row in cursor.fetchall()
    ]
    cursor.close()
    conn.close()

    # 3) Build normalized→info map
    norm_to_info = {
        normalize_name(ci['name']): ci
        for ci in cols_info
    }

    # 4) Map CSV headers → real column names
    col_map = {}
    for hdr in df.columns:
        norm = normalize_name(hdr)
        if norm in norm_to_info:
            info = norm_to_info[norm]
        else:
            # substring fallback
            cands = [ci for n, ci in norm_to_info.items() if norm in n]
            if len(cands) == 1:
                info = cands[0]
            else:
                raise ValueError(
                    f"Cannot map CSV column '{hdr}'. "
                    f"Candidates: {[ci['name'] for ci in cands]}"
                )
        col_map[hdr] = info['name']
    df = df.rename(columns=col_map)
    real_cols = list(col_map.values())

    # 5) Prepare a row‐by‐row converter that handles NULLs
    def make_converter(ci):
        name, dtype, prec, scale = ci['name'], ci['dtype'], ci['prec'], ci['scale']
        if dtype in _DATE_TYPES:
            def conv(v):
                if v == '':
                    return None
                dt = pd.to_datetime(v, dayfirst=dayfirst, errors='raise')
                return dt.date() if dtype == 'date' else dt
        elif dtype in _INT_TYPES:
            def conv(v):
                if v == '':
                    return None
                return int(v)
        elif dtype in _FLOAT_TYPES:
            def conv(v):
                if v == '':
                    return None
                return float(v)
        elif dtype in _DEC_TYPES:
            # quantize to the table’s scale
            quantum = Decimal((0, (1,), -scale))
            def conv(v):
                if v == '':
                    return None
                d = Decimal(v)
                return d.quantize(quantum)
        else:
            def conv(v):
                # leave strings and any other types untouched
                return v if v != '' else None

        return conv

    converters = {
        ci['name']: make_converter(ci)
        for ci in cols_info
        if ci['name'] in real_cols
    }

    # 6) Build INSERT SQL
    col_list = ", ".join(f"[{c}]" for c in real_cols)
    ph       = ", ".join("?" for _ in real_cols)
    insert_sql = f"INSERT INTO {table_name} ({col_list}) VALUES ({ph})"

    # 7) Open fresh connection & bulk‐insert
    conn = pyodbc.connect(conn_str, timeout=5)
    cursor = conn.cursor()
    cursor.fast_executemany = True

    # Convert & collect rows
    data = []
    for _, row in df.iterrows():
        converted = []
        for col in real_cols:
            val = row[col]
            converted.append(converters[col](val))
        data.append(tuple(converted))

    cursor.executemany(insert_sql, data)
    conn.commit()
    cursor.close()
    conn.close()

    print(f"✅ Appended {len(data)} rows to {table_name}.")


In [51]:
# Install ODBC Driver 17 for SQL Server and pyodbc if not already installed
# For Ubuntu-based Codespaces
!sudo ACCEPT_EULA=Y apt-get update
!sudo ACCEPT_EULA=Y apt-get install -y curl apt-transport-https gnupg
!curl https://packages.microsoft.com/keys/microsoft.asc | sudo apt-key add -
!curl https://packages.microsoft.com/config/ubuntu/$(lsb_release -rs)/prod.list | sudo tee /etc/apt/sources.list.d/mssql-release.list
!sudo apt-get update
!sudo ACCEPT_EULA=Y apt-get install -y msodbcsql17
%pip install pyodbc

# Cell 2: Example usage
csv_path = "/workspaces/vikrant_dubai/SQL/D24hparameter_rounded.csv"
table_name = "dbo.D24hparameter_rounded"
server = "4.255.48.194,1433"
database = "MOCCAE"
username = "devuser"
password = "Vikr@1234"


append_csv_to_sqlserver(
    csv_path=csv_path,
    table_name=table_name,
    server=server,
    database=database,
    username=username,
    password=password
)


Hit:1 https://packages.microsoft.com/repos/microsoft-ubuntu-focal-prod focal InRelease
Hit:2 https://packages.microsoft.com/ubuntu/20.04/prod focal InRelease         
Hit:3 https://dl.yarnpkg.com/debian stable InRelease                           
Hit:4 https://repo.anaconda.com/pkgs/misc/debrepo/conda stable InRelease       
Hit:5 http://archive.ubuntu.com/ubuntu focal InRelease                         
Hit:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease                 
Hit:7 http://security.ubuntu.com/ubuntu focal-security InRelease               
Hit:8 http://archive.ubuntu.com/ubuntu focal-backports InRelease    
Hit:9 https://packagecloud.io/github/git-lfs/ubuntu focal InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency tree       
Reading state information... Done
curl is already the newest version (7.68.0-1ubuntu2.25).
gnupg is already the newest version (2.2.19-3ubuntu2.4).
apt-transport-https is already the newest version (2.0.1

In [52]:
# Install ODBC Driver 17 for SQL Server and pyodbc if not already installed
# For Ubuntu-based Codespaces
!sudo ACCEPT_EULA=Y apt-get update
!sudo ACCEPT_EULA=Y apt-get install -y curl apt-transport-https gnupg
!curl https://packages.microsoft.com/keys/microsoft.asc | sudo apt-key add -
!curl https://packages.microsoft.com/config/ubuntu/$(lsb_release -rs)/prod.list | sudo tee /etc/apt/sources.list.d/mssql-release.list
!sudo apt-get update
!sudo ACCEPT_EULA=Y apt-get install -y msodbcsql17
%pip install pyodbc

# Cell 2: Example usage
csv_path = "/workspaces/vikrant_dubai/SQL/D24hparameter.csv"
table_name = "dbo.D24hparameter"
server = "4.255.48.194,1433"
database = "MOCCAE"
username = "devuser"
password = "Vikr@1234"

append_csv_to_sqlserver(
    csv_path=csv_path,
    table_name=table_name,
    server=server,
    database=database,
    username=username,
    password=password
)


Hit:1 https://dl.yarnpkg.com/debian stable InRelease
Hit:2 https://repo.anaconda.com/pkgs/misc/debrepo/conda stable InRelease       
Hit:3 https://packages.microsoft.com/repos/microsoft-ubuntu-focal-prod focal InRelease
Hit:4 https://packages.microsoft.com/ubuntu/20.04/prod focal InRelease         
0% [Waiting for headers] [Connecting to security.ubuntu.com (185.125.190.82)] [

Hit:5 http://archive.ubuntu.com/ubuntu focal InRelease                         
Hit:6 http://security.ubuntu.com/ubuntu focal-security InRelease               
Hit:7 http://archive.ubuntu.com/ubuntu focal-updates InRelease                 
Hit:8 http://archive.ubuntu.com/ubuntu focal-backports InRelease       
Hit:9 https://packagecloud.io/github/git-lfs/ubuntu focal InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency tree       
Reading state information... Done
curl is already the newest version (7.68.0-1ubuntu2.25).
gnupg is already the newest version (2.2.19-3ubuntu2.4).
apt-transport-https is already the newest version (2.0.10).
0 upgraded, 0 newly installed, 0 to remove and 42 not upgraded.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   983  100   983    0     0  23404      0 --:--:-- --:--:-- --:--:-- 23404
OK
  % Total    % R

In [53]:
# Install ODBC Driver 17 for SQL Server and pyodbc if not already installed
# For Ubuntu-based Codespaces
!sudo ACCEPT_EULA=Y apt-get update
!sudo ACCEPT_EULA=Y apt-get install -y curl apt-transport-https gnupg
!curl https://packages.microsoft.com/keys/microsoft.asc | sudo apt-key add -
!curl https://packages.microsoft.com/config/ubuntu/$(lsb_release -rs)/prod.list | sudo tee /etc/apt/sources.list.d/mssql-release.list
!sudo apt-get update
!sudo ACCEPT_EULA=Y apt-get install -y msodbcsql17
%pip install pyodbc

# Cell 2: Example usage
csv_path = "/workspaces/vikrant_dubai/SQL/Finder_table.csv"
table_name = "dbo.Finder_table"
server = "4.255.48.194,1433"
database = "MOCCAE"
username = "devuser"
password = "Vikr@1234"

append_csv_to_sqlserver(
    csv_path=csv_path,
    table_name=table_name,
    server=server,
    database=database,
    username=username,
    password=password
)


Hit:1 https://packages.microsoft.com/repos/microsoft-ubuntu-focal-prod focal InRelease
Hit:2 https://packages.microsoft.com/ubuntu/20.04/prod focal InRelease         
Hit:3 https://dl.yarnpkg.com/debian stable InRelease                           
Hit:4 https://repo.anaconda.com/pkgs/misc/debrepo/conda stable InRelease       
Get:5 http://security.ubuntu.com/ubuntu focal-security InRelease [128 kB]      
Hit:6 http://archive.ubuntu.com/ubuntu focal InRelease                         
Get:7 http://archive.ubuntu.com/ubuntu focal-updates InRelease [128 kB]        
Hit:8 https://packagecloud.io/github/git-lfs/ubuntu focal InRelease            
Get:9 http://archive.ubuntu.com/ubuntu focal-backports InRelease [128 kB]
Get:10 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 Packages [4908 kB]
Fetched 5291 kB in 4s (1412 kB/s)    
Reading package lists... Done
Reading package lists... Done
Building dependency tree       
Reading state information... Done
curl is already the newest vers

In [54]:
# Install ODBC Driver 17 for SQL Server and pyodbc if not already installed
# For Ubuntu-based Codespaces
!sudo ACCEPT_EULA=Y apt-get update
!sudo ACCEPT_EULA=Y apt-get install -y curl apt-transport-https gnupg
!curl https://packages.microsoft.com/keys/microsoft.asc | sudo apt-key add -
!curl https://packages.microsoft.com/config/ubuntu/$(lsb_release -rs)/prod.list | sudo tee /etc/apt/sources.list.d/mssql-release.list
!sudo apt-get update
!sudo ACCEPT_EULA=Y apt-get install -y msodbcsql17
%pip install pyodbc

# Cell 2: Example usage
csv_path = "/workspaces/vikrant_dubai/SQL/KPI_NS.csv"
table_name = "dbo.KPI_NS"
server = "4.255.48.194,1433"
database = "MOCCAE"
username = "devuser"
password = "Vikr@1234"

append_csv_to_sqlserver(
    csv_path=csv_path,
    table_name=table_name,
    server=server,
    database=database,
    username=username,
    password=password
)


Hit:1 https://dl.yarnpkg.com/debian stable InRelease
Hit:2 https://packages.microsoft.com/repos/microsoft-ubuntu-focal-prod focal InRelease
Hit:3 https://packages.microsoft.com/ubuntu/20.04/prod focal InRelease         
Hit:4 https://repo.anaconda.com/pkgs/misc/debrepo/conda stable InRelease       
Hit:5 http://security.ubuntu.com/ubuntu focal-security InRelease               
Hit:6 http://archive.ubuntu.com/ubuntu focal InRelease                         
Hit:7 http://archive.ubuntu.com/ubuntu focal-updates InRelease        
Hit:9 http://archive.ubuntu.com/ubuntu focal-backports InRelease      
Hit:8 https://packagecloud.io/github/git-lfs/ubuntu focal InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency tree       
Reading state information... Done
curl is already the newest version (7.68.0-1ubuntu2.25).
gnupg is already the newest version (2.2.19-3ubuntu2.4).
apt-transport-https is already the newest version (2.0.10).
0 upgraded, 0 newly installed,

In [55]:
# Install ODBC Driver 17 for SQL Server and pyodbc if not already installed
# For Ubuntu-based Codespaces
!sudo ACCEPT_EULA=Y apt-get update
!sudo ACCEPT_EULA=Y apt-get install -y curl apt-transport-https gnupg
!curl https://packages.microsoft.com/keys/microsoft.asc | sudo apt-key add -
!curl https://packages.microsoft.com/config/ubuntu/$(lsb_release -rs)/prod.list | sudo tee /etc/apt/sources.list.d/mssql-release.list
!sudo apt-get update
!sudo ACCEPT_EULA=Y apt-get install -y msodbcsql17
%pip install pyodbc

# Cell 2: Example usage
csv_path = "/workspaces/vikrant_dubai/SQL/KPI_PM.csv"
table_name = "dbo.KPI_PM"
server = "4.255.48.194,1433"
database = "MOCCAE"
username = "devuser"
password = "Vikr@1234"

append_csv_to_sqlserver(
    csv_path=csv_path,
    table_name=table_name,
    server=server,
    database=database,
    username=username,
    password=password
)


Hit:1 https://dl.yarnpkg.com/debian stable InRelease
Hit:2 https://packages.microsoft.com/repos/microsoft-ubuntu-focal-prod focal InRelease
Hit:3 https://packages.microsoft.com/ubuntu/20.04/prod focal InRelease         
Hit:4 https://repo.anaconda.com/pkgs/misc/debrepo/conda stable InRelease       
Hit:5 http://archive.ubuntu.com/ubuntu focal InRelease                         
Hit:6 http://security.ubuntu.com/ubuntu focal-security InRelease               
Hit:7 http://archive.ubuntu.com/ubuntu focal-updates InRelease        
Hit:9 http://archive.ubuntu.com/ubuntu focal-backports InRelease      
Hit:8 https://packagecloud.io/github/git-lfs/ubuntu focal InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency tree       
Reading state information... Done
curl is already the newest version (7.68.0-1ubuntu2.25).
gnupg is already the newest version (2.2.19-3ubuntu2.4).
apt-transport-https is already the newest version (2.0.10).
0 upgraded, 0 newly installed,

In [56]:
# Install ODBC Driver 17 for SQL Server and pyodbc if not already installed
# For Ubuntu-based Codespaces
!sudo ACCEPT_EULA=Y apt-get update
!sudo ACCEPT_EULA=Y apt-get install -y curl apt-transport-https gnupg
!curl https://packages.microsoft.com/keys/microsoft.asc | sudo apt-key add -
!curl https://packages.microsoft.com/config/ubuntu/$(lsb_release -rs)/prod.list | sudo tee /etc/apt/sources.list.d/mssql-release.list
!sudo apt-get update
!sudo ACCEPT_EULA=Y apt-get install -y msodbcsql17
%pip install pyodbc

# Cell 2: Example usage
csv_path = "/workspaces/vikrant_dubai/SQL/KPI_WHO.csv"
table_name = "dbo.KPI_WHO"
server = "4.255.48.194,1433"
database = "MOCCAE"
username = "devuser"
password = "Vikr@1234"

append_csv_to_sqlserver(
    csv_path=csv_path,
    table_name=table_name,
    server=server,
    database=database,
    username=username,
    password=password
)


Hit:1 https://packages.microsoft.com/repos/microsoft-ubuntu-focal-prod focal InRelease
Hit:2 https://dl.yarnpkg.com/debian stable InRelease                           
Hit:3 https://packages.microsoft.com/ubuntu/20.04/prod focal InRelease         
Hit:4 https://repo.anaconda.com/pkgs/misc/debrepo/conda stable InRelease       
Hit:5 http://security.ubuntu.com/ubuntu focal-security InRelease               
Hit:6 http://archive.ubuntu.com/ubuntu focal InRelease                         
Hit:7 http://archive.ubuntu.com/ubuntu focal-updates InRelease        
Hit:9 http://archive.ubuntu.com/ubuntu focal-backports InRelease
Hit:8 https://packagecloud.io/github/git-lfs/ubuntu focal InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency tree       
Reading state information... Done
curl is already the newest version (7.68.0-1ubuntu2.25).
gnupg is already the newest version (2.2.19-3ubuntu2.4).
apt-transport-https is already the newest version (2.0.10).
0 upgrade

In [57]:
# Install ODBC Driver 17 for SQL Server and pyodbc if not already installed
# For Ubuntu-based Codespaces
!sudo ACCEPT_EULA=Y apt-get update
!sudo ACCEPT_EULA=Y apt-get install -y curl apt-transport-https gnupg
!curl https://packages.microsoft.com/keys/microsoft.asc | sudo apt-key add -
!curl https://packages.microsoft.com/config/ubuntu/$(lsb_release -rs)/prod.list | sudo tee /etc/apt/sources.list.d/mssql-release.list
!sudo apt-get update
!sudo ACCEPT_EULA=Y apt-get install -y msodbcsql17
%pip install pyodbc

# Cell 2: Example usage
csv_path = "/workspaces/vikrant_dubai/SQL/STdetails.csv"
table_name = "dbo.STdetails"
server = "4.255.48.194,1433"
database = "MOCCAE"
username = "devuser"
password = "Vikr@1234"

append_csv_to_sqlserver(
    csv_path=csv_path,
    table_name=table_name,
    server=server,
    database=database,
    username=username,
    password=password
)


Hit:1 https://dl.yarnpkg.com/debian stable InRelease
Hit:2 https://packages.microsoft.com/repos/microsoft-ubuntu-focal-prod focal InRelease
Hit:3 https://packages.microsoft.com/ubuntu/20.04/prod focal InRelease         
0% [Connecting to archive.ubuntu.com (185.125.190.83)] [Connecting to security.

Hit:4 https://repo.anaconda.com/pkgs/misc/debrepo/conda stable InRelease       
Hit:5 http://archive.ubuntu.com/ubuntu focal InRelease                         
Hit:6 http://security.ubuntu.com/ubuntu focal-security InRelease               
Hit:7 http://archive.ubuntu.com/ubuntu focal-updates InRelease        
Hit:9 http://archive.ubuntu.com/ubuntu focal-backports InRelease      
Hit:8 https://packagecloud.io/github/git-lfs/ubuntu focal InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency tree       
Reading state information... Done
curl is already the newest version (7.68.0-1ubuntu2.25).
gnupg is already the newest version (2.2.19-3ubuntu2.4).
apt-transport-https is already the newest version (2.0.10).
0 upgraded, 0 newly installed, 0 to remove and 42 not upgraded.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   983  100   983    0   